# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import glob
import json
import os
import pandas as pd
import psycopg2


from sql_queries import create_table_queries, drop_table_queries, \
    song_table_insert, artist_table_insert, time_table_insert, user_table_insert, time_table_insert, songplay_table_insert, \
    song_select

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")
song_files[:5]

['C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json']

In [5]:
filepath = song_files[0]
filepath

'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json'

In [6]:
import json

with open(filepath) as fio:
    list_of_dicts = [ # wee need a root -> wrap in list
        json.load(fio)
    ] 

list_of_dicts

[{'num_songs': 1,
  'artist_id': 'ARD7TVE1187B99BFB1',
  'artist_latitude': None,
  'artist_longitude': None,
  'artist_location': 'California - LA',
  'artist_name': 'Casual',
  'song_id': 'SOMZWCG12A8C13C480',
  'title': "I Didn't Mean To",
  'duration': 218.93179,
  'year': 0}]

In [7]:
# converting json dataset from dictionary to dataframe
df = pd.DataFrame.from_records(list_of_dicts)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = [
    tuple(x) for x in df.loc[:,["song_id", "title", "artist_id", "year", "duration"]].to_numpy()
]

song_data

[('SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179)]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
song_table_insert

'INSERT INTO songs(song_id, title, artist_id, year, duration)                         VALUES(%s, %s, %s, %s, %s)\n                        ON CONFLICT DO NOTHING;'

In [10]:
cur.execute(song_table_insert, song_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = [
    tuple(x) for x in df.loc[:,["artist_id", 
                                           "artist_name",
                                           "artist_location",
                                           "artist_latitude",
                                           "artist_longitude"]].to_numpy()
]

artist_data

[('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
artist_table_insert

'INSERT INTO artists(artist_id, name, location, latitude, longitude)                         VALUES(%s, %s, %s, %s, %s)\n                        ON CONFLICT DO NOTHING;'

In [13]:
conn.rollback()
cur.execute(artist_table_insert, artist_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = song_files = get_files("data/log_data")
log_files[:5]

['C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\log_data\\2018\\11\\2018-11-01-events.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\log_data\\2018\\11\\2018-11-02-events.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\log_data\\2018\\11\\2018-11-03-events.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\log_data\\2018\\11\\2018-11-04-events.json',
 'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\log_data\\2018\\11\\2018-11-05-events.json']

In [15]:
filepath = log_files[0]
filepath

'C:\\Users\\marcus\\Documents\\_Working\\DatEng1\\data\\log_data\\2018\\11\\2018-11-01-events.json'

In [16]:
# Since the json has multiple rows, we need a root -> wrap in list, see https://stackoverflow.com/a/51830719

list_of_dicts = [
    json.loads(line) for line in open(filepath, 'r')
] 
list_of_dicts

[{'artist': None,
  'auth': 'Logged In',
  'firstName': 'Walter',
  'gender': 'M',
  'itemInSession': 0,
  'lastName': 'Frye',
  'length': None,
  'level': 'free',
  'location': 'San Francisco-Oakland-Hayward, CA',
  'method': 'GET',
  'page': 'Home',
  'registration': 1540919166796.0,
  'sessionId': 38,
  'song': None,
  'status': 200,
  'ts': 1541105830796,
  'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"',
  'userId': '39'},
 {'artist': None,
  'auth': 'Logged In',
  'firstName': 'Kaylee',
  'gender': 'F',
  'itemInSession': 0,
  'lastName': 'Summers',
  'length': None,
  'level': 'free',
  'location': 'Phoenix-Mesa-Scottsdale, AZ',
  'method': 'GET',
  'page': 'Home',
  'registration': 1540344794796.0,
  'sessionId': 139,
  'song': None,
  'status': 200,
  'ts': 1541106106796,
  'userAgent': '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safar

In [17]:
df = pd.DataFrame.from_records(list_of_dicts)
df = df.loc[df.page=="NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
t = pd.to_datetime(df["ts"], unit="ms")
t

2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [19]:
df["ts2"] = pd.to_datetime(df["ts"], unit="ms")

In [20]:
#timestamp, hour, day, week of year, month, year, and weekday
time_data = [(r, r.hour, r.day, r.weekofyear, r.month, r.year, r.weekday())
              for r in t]

column_labels = ["Timestamp", "Hour", "Day", "Weekofyear", "Month", "Year", "Weekday"]

In [21]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

,Timestamp,Hour,Day,Weekofyear,Month,Year,Weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,3
1,2018-11-01 21:05:52.796,21,1,44,11,2018,3
2,2018-11-01 21:08:16.796,21,1,44,11,2018,3
3,2018-11-01 21:11:13.796,21,1,44,11,2018,3
4,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
time_table_insert

'INSERT INTO times(start_time, hour, day, week, month, year, weekday)                         VALUES(%s, %s, %s, %s, %s, %s, %s)\n                        ON CONFLICT DO NOTHING;'

In [23]:
conn.rollback()
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df.loc[:,['userId','firstName','lastName','gender','level']].drop_duplicates()
user_df

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
user_table_insert

'INSERt INTO users(user_id, first_name, last_name, gender, level)\n                        VALUES(%s, %s, %s, %s, %s)\n                        ON CONFLICT DO NOTHING;'

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
df.loc[:,["song", "artist", "length"]].head()

,song,artist,length
2,You Gotta Be,Des'ree,246.30812
4,Flat 55,Mr Oizo,144.03873
5,Quem Quiser Encontrar O Amor,Tamba Trio,177.18812
6,Eriatarka,The Mars Volta,380.42077
7,Becoming Insane,Infected Mushroom,440.26730


In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
        # if no match (song title, artist name, song length) exists, insert row in songplay with NONE as ID
    songplay_data = (row.ts2, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    #print(songplay_data)

    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.